In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# SHP 파일 경로 설정
shp_file = 'sig.shp'

# CSV 파일 경로 설정
csv_file = 'up_down.csv'

# SHP 파일 읽기
data = gpd.read_file(shp_file)

# CSV 파일 읽기
df = pd.read_csv(csv_file, encoding='cp949')

# NaN 값 처리
df['행정구역별'].fillna('', inplace=True)
df['증감율'].fillna(-9999, inplace=True)  # 결측값을 -9999로 대체

# 데이터 조인 (순서대로 조인)
data_with_color = data.merge(df[['행정구역별', '증감율']], left_index=True, right_index=True, how='left')

# 불필요한 컬럼 삭제
data_with_color.drop(['행정구역별'], axis=1, inplace=True)

# 증감율 컬럼 형식 변환
data_with_color['증감율'] = data_with_color['증감율'].astype(float)

# 데이터가 있는지 확인 후 시각화
if not data_with_color['증감율'].isnull().all():
    fig, ax = plt.subplots(figsize=(10, 10))

    # 행정구역별 경계 추가
    data.boundary.plot(ax=ax, color='lightgray', linewidth=0.5)

    # 시 단위로 구역을 필터링하여 경계 표시
    for 시 in data_with_color['SIG_CD'].unique():
        filtered_data = data_with_color[data_with_color['SIG_CD'] == 시]
        filtered_data.boundary.plot(ax=ax, color='lightgray', linewidth=1.5)

    # 증감율에 따라 색상 부여
    data_with_color['color'] = np.where(data_with_color['증감율'] < -1024, 'lightgray',
                                        np.where(data_with_color['증감율'] < -100, 'salmon',
                                                 np.where(data_with_color['증감율'] < -50, 'red',
                                                          np.where(data_with_color['증감율'] <= 0, 'mistyrose',
                                                                   np.where(data_with_color['증감율'] < 50, 'lightskyblue',
                                                                            np.where(data_with_color['증감율'] < 100, 'deepskyblue',
                                                                                     'darkblue'))))))
    data_with_color['color'].fillna('yellow', inplace=True)

    # 그래프 출력
    data_with_color.plot(ax=ax, color=data_with_color['color'], legend=True)

    # 그래프 출력 설정
    ax.set_axis_off()

    # 그래프 제목 설정
    ax.set_title('change rate', fontweight='bold', fontsize=16)

    # 그래프 출력
    plt.show()
else:
    print("색상 컬럼에 대한 데이터가 없습니다.")

print(data_with_color)




DriverError: 'sig.shp' does not exist in the file system, and is not recognized as a supported dataset name.